In [10]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# prepare dataset

batch_size = 64
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])  # 归一化,均值和方差

train_dataset = datasets.MNIST(
    root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(
    root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# design model using class


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 卷积层
        self.conv1 = torch.nn.Conv2d(1, 5, padding=1, kernel_size=3)
        self.conv2 = torch.nn.Conv2d(5, 10, kernel_size=3)
        self.conv3 = torch.nn.Conv2d(10, 15, kernel_size=3)
        # 池化层
        self.pooling = torch.nn.MaxPool2d(2)
        # 全连接层
        self.l1 = torch.nn.Linear(60, 1024)
        self.l2 = torch.nn.Linear(1024, 512)
        self.l5 = torch.nn.Linear(512, 10)

    def forward(self, x):
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = F.relu(self.pooling(self.conv3(x)))
        x = x.view(batch_size, -1)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = self.l5(x)
        return x  # 最后一层不做激活，不进行非线性变换


model = Net()

# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# training cycle forward, backward, update


def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        # 获得一个批次的数据和标签
        inputs, target = data

        optimizer.zero_grad()
        # 获得模型预测结果(64, 10)
        outputs = model(inputs)
        # 交叉熵代价函数outputs(64,10),target（64）
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' %
                  (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0


def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data

            outputs = model(images)
            # dim = 1 列是第0个维度，行是第1个维度
            _, predicted = torch.max(outputs.data, dim=1)
            # print("predicted = ",predicted,"\tlabel = ",labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # 张量之间的比较运算
    print('accuracy on test set: %d %% ' % (100*correct/total))


if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()


[1,   300] loss: 2.272
[1,   600] loss: 0.801
[1,   900] loss: 0.286
accuracy on test set: 91 % 
[2,   300] loss: 0.186
[2,   600] loss: 0.155
[2,   900] loss: 0.139
accuracy on test set: 95 % 
[3,   300] loss: 0.110
[3,   600] loss: 0.106
[3,   900] loss: 0.097
accuracy on test set: 97 % 
[4,   300] loss: 0.079
[4,   600] loss: 0.087
[4,   900] loss: 0.076
accuracy on test set: 97 % 
[5,   300] loss: 0.068
[5,   600] loss: 0.068
[5,   900] loss: 0.071
accuracy on test set: 97 % 
[6,   300] loss: 0.063
[6,   600] loss: 0.059
[6,   900] loss: 0.057
accuracy on test set: 98 % 
[7,   300] loss: 0.053
[7,   600] loss: 0.051
[7,   900] loss: 0.051
accuracy on test set: 98 % 
[8,   300] loss: 0.046
[8,   600] loss: 0.048
[8,   900] loss: 0.050
accuracy on test set: 98 % 
[9,   300] loss: 0.041
[9,   600] loss: 0.043
[9,   900] loss: 0.040
accuracy on test set: 98 % 
[10,   300] loss: 0.035
[10,   600] loss: 0.043
[10,   900] loss: 0.038
accuracy on test set: 98 % 
